# Limpieza de datos

### Importacion de bibliotecas

In [29]:
from pathlib import Path
import pandas as pd
import numpy as np

### Carga de datos

Agregamos `low_memory=false` como parametro de la funcion `pd.read_csv()`

In [30]:
df = pd.read_csv("..\..\..\DataSet\DataBase.csv", low_memory=False)

### Identificacion de datos

In [31]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1401331 entries, 0 to 1401330
Data columns (total 19 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   ao_hechos             1400873 non-null  float64
 1   mes_hechos            1400873 non-null  object 
 2   fecha_hechos          1400873 non-null  object 
 3   ao_inicio             1401331 non-null  int64  
 4   mes_inicio            1401331 non-null  object 
 5   fecha_inicio          1401328 non-null  object 
 6   delito                1401331 non-null  object 
 7   fiscalia              1401329 non-null  object 
 8   agencia               1401331 non-null  object 
 9   unidad_investigacion  1401104 non-null  object 
 10  categoria_delito      1401331 non-null  object 
 11  calle_hechos          1397390 non-null  object 
 12  calle_hechos2         539997 non-null   object 
 13  colonia_hechos        1340993 non-null  object 
 14  alcaldia_hechos       1397166 non-

Con `.value_counts` sabremos cuantos registros tenemos   

In [32]:
print(df['ao_hechos'].value_counts(dropna=False))

2018.0    254383
2019.0    244563
2017.0    227453
2021.0    218659
2020.0    203033
           ...  
1954.0         1
1906.0         1
1930.0         1
1942.0         1
1960.0         1
Name: ao_hechos, Length: 76, dtype: int64


Eliminar columnas que no resultan utilies con `.drop`

In [39]:
dfL = df.drop([
            'tempo', 
            'ao_inicio', 
            'mes_inicio', 
            'fecha_inicio', 
            'agencia', 
            'unidad_investigacion',
            'competencia'] , axis=1)


se borran los compos "NA" de la columna longitud pero se deja el espacio en blnaco

In [40]:
dfL.drop(dfL[dfL['longitud'] == 'NA'].index, inplace = True)

Se eliminan las filas con el espacio en blnaco

In [41]:
dfL = dfL[df['longitud'].notna()]

Confirmamos que se hayan eliminado, imprimeindo las columnas de la base, esto con `print(df.columns)`

In [42]:
print(df.columns)

Index(['ao_hechos', 'mes_hechos', 'fecha_hechos', 'ao_inicio', 'mes_inicio',
       'fecha_inicio', 'delito', 'fiscalia', 'agencia', 'unidad_investigacion',
       'categoria_delito', 'calle_hechos', 'calle_hechos2', 'colonia_hechos',
       'alcaldia_hechos', 'competencia', 'longitud', 'latitud', 'tempo'],
      dtype='object')


### Separación de fecha en dia y hora

Nos interesa saber que dia fue el delito asi como la hora en la que se sutito, tenemos estos datos en la columna "fecha_hechos" pero debemos separarlos

In [43]:
dfT = dfL["fecha_hechos"].str.split('[- ]', expand=True)

Esto nos genera un nuevo dataframe pero con numeros como nombres de columnas por lo que debemos cambiarlos

In [45]:
dfT = dfT.rename(columns={1 :'dia_hechos', 3 :'hora_hechos'})

print(dfT)

            0 dia_hechos   2 hora_hechos
0        2016         01  31    22:16:00
1        2016         01  31    20:50:00
2        2016         02  01    00:30:00
3        2016         01  31    22:00:00
4        2015         12  25    12:00:00
...       ...        ...  ..         ...
1401326  2022         02  28    06:26:00
1401327  2022         02  28    12:00:00
1401328  2021         05  05    09:00:00
1401329  2022         02  20    15:50:00
1401330  2022         02  02    12:40:00

[1341941 rows x 4 columns]


Eliminamos las columnas que no necesitamos

In [46]:
dfT = dfT.drop([0,2] , axis=1)
print(dfT)

        dia_hechos hora_hechos
0               01    22:16:00
1               01    20:50:00
2               02    00:30:00
3               01    22:00:00
4               12    12:00:00
...            ...         ...
1401326         02    06:26:00
1401327         02    12:00:00
1401328         05    09:00:00
1401329         02    15:50:00
1401330         02    12:40:00

[1341941 rows x 2 columns]


Ahora agregaremos estas dos columnas a nuestro dataframe principal esto con `.concat`

In [49]:
dfS= pd.concat([dfL ,dfT], axis=1)

Eliminaremos la columna "fecha:hehcos" pues ya no es relevante y acomodaremos las columnas agregadas para que sea de la forma dd/mm/aaaa

In [50]:
dfS = dfS.drop(['fecha_hechos'] , axis=1)

dfS = dfS[['dia_hechos', 'mes_hechos', 'ao_hechos', 'hora_hechos', 'delito', 'fiscalia', 'categoria_delito',
           'calle_hechos', 'calle_hechos2', 'colonia_hechos', 'alcaldia_hechos', 'longitud', 'latitud']]
print(dfS)

        dia_hechos mes_hechos  ao_hechos hora_hechos  \
0               01      Enero     2016.0    22:16:00   
1               01      Enero     2016.0    20:50:00   
2               02    Febrero     2016.0    00:30:00   
3               01      Enero     2016.0    22:00:00   
4               12  Diciembre     2015.0    12:00:00   
...            ...        ...        ...         ...   
1401326         02    Febrero     2022.0    06:26:00   
1401327         02    Febrero     2022.0    12:00:00   
1401328         05       Mayo     2021.0    09:00:00   
1401329         02    Febrero     2022.0    15:50:00   
1401330         02    Febrero     2022.0    12:40:00   

                                                    delito  \
0                      DAÑO EN PROPIEDAD AJENA INTENCIONAL   
1        ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...   
2                             NARCOMENUDEO POSESION SIMPLE   
3           ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA   
4                

### Filtar zonas de estuido

El siguinte bloque de codigo lo que hara sera filtrar las zonas de estudio de nuestro dataframe principal para despues crearle uno a cada zona

In [51]:
dfA = dfS.copy()


dfIZTAPALAPA= dfA.copy()
dfIZTAPALAPA.drop(dfIZTAPALAPA[dfIZTAPALAPA['alcaldia_hechos'] != 'IZTAPALAPA'].index, inplace = True)  #filtramos las alcaldias por iztapalapa
dfIZTAPALAPA = dfIZTAPALAPA[dfIZTAPALAPA['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

dfGUSTAVOAMADER = dfA.copy()
dfGUSTAVOAMADER.drop(dfGUSTAVOAMADER [dfGUSTAVOAMADER ['alcaldia_hechos'] != 'GUSTAVO A MADERO'].index, inplace = True)  #filtramos las alcaldias por Gustavo A Madero
dfGUSTAVOAMADER = dfGUSTAVOAMADER [dfGUSTAVOAMADER ['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

dfAZCAPOTZALCO = dfA.copy()
dfAZCAPOTZALCO.drop(dfAZCAPOTZALCO [dfAZCAPOTZALCO ['alcaldia_hechos'] != 'AZCAPOTZALCO'].index, inplace = True)  #filtramos las alcaldias por Azcapotzalco
dfAZCAPOTZALCO = dfAZCAPOTZALCO [dfAZCAPOTZALCO ['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

dfVENUSTIANOCARRANZA = dfA.copy()
dfVENUSTIANOCARRANZA.drop(dfVENUSTIANOCARRANZA [dfVENUSTIANOCARRANZA ['alcaldia_hechos'] != 'VENUSTIANO CARRANZA'].index, inplace = True)  #filtramos las alcaldias por Venustiano Carranza
dfVENUSTIANOCARRANZA = dfVENUSTIANOCARRANZA [dfVENUSTIANOCARRANZA ['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

dfCUAUHTEMOC = dfA.copy()
dfCUAUHTEMOC.drop(dfCUAUHTEMOC [dfCUAUHTEMOC ['alcaldia_hechos'] != 'CUAUHTEMOC'].index, inplace = True)  #filtramos las alcaldias por Cuauhtemoc
dfCUAUHTEMOC = dfCUAUHTEMOC [dfCUAUHTEMOC ['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

dfIZTACALCO = dfA.copy()
dfIZTACALCO.drop(dfIZTACALCO [dfIZTACALCO ['alcaldia_hechos'] != 'IZTACALCO'].index, inplace = True)  #filtramos las alcaldias por Iztacalco
dfIZTACALCO = dfIZTACALCO [dfIZTACALCO ['alcaldia_hechos'].notna()] #se eliminan las filas con el espacio en blnaco

Juntaremos todas las zonas de estudio en un solo dataframe que volvera a ser nuestro principal

In [53]:
dfFinal = pd.concat([dfIZTAPALAPA,dfGUSTAVOAMADER,dfAZCAPOTZALCO,dfVENUSTIANOCARRANZA,dfCUAUHTEMOC,dfIZTACALCO])
print(dfFinal)

        dia_hechos mes_hechos  ao_hechos hora_hechos  \
1               01      Enero     2016.0    20:50:00   
2               02    Febrero     2016.0    00:30:00   
3               01      Enero     2016.0    22:00:00   
17              01      Enero     2016.0    05:45:00   
19              02    Febrero     2016.0    07:45:00   
...            ...        ...        ...         ...   
1401299         02    Febrero     2022.0    09:00:00   
1401301         02    Febrero     2022.0    12:00:00   
1401309         02    Febrero     2022.0    13:00:00   
1401315         02    Febrero     2022.0    00:45:00   
1401327         02    Febrero     2022.0    12:00:00   

                                                    delito  \
1        ROBO DE VEHICULO DE SERVICIO PARTICULAR CON VI...   
2                             NARCOMENUDEO POSESION SIMPLE   
3           ROBO A TRANSEUNTE EN VIA PUBLICA CON VIOLENCIA   
17       ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...   
19               

### Filtrar años de interes

Este bloque de codigo lo que hará es que eliminara los las filas de los años menores a 2016 y mayores a 2021

In [54]:
dfFinal.drop(dfFinal[dfFinal['ao_hechos'] <= 2015 ].index, inplace = True)  

dfFinal = dfFinal[dfFinal['ao_hechos'].notna()] 

dfFinal.drop(dfFinal[dfFinal['ao_hechos'] >= 2022 ].index, inplace = True)  

dfFinal = dfFinal[dfFinal['ao_hechos'].notna()] 

Con esto nuestra base estaria limpia según neustras necesidades, podemos hacer un ultimo chequeo y ver como es que quedo conformada

In [55]:
print(dfFinal.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714383 entries, 1 to 1400430
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dia_hechos        714383 non-null  object 
 1   mes_hechos        714383 non-null  object 
 2   ao_hechos         714383 non-null  float64
 3   hora_hechos       714383 non-null  object 
 4   delito            714383 non-null  object 
 5   fiscalia          714383 non-null  object 
 6   categoria_delito  714383 non-null  object 
 7   calle_hechos      713254 non-null  object 
 8   calle_hechos2     293996 non-null  object 
 9   colonia_hechos    713800 non-null  object 
 10  alcaldia_hechos   714383 non-null  object 
 11  longitud          714383 non-null  float64
 12  latitud           714383 non-null  float64
dtypes: float64(3), object(10)
memory usage: 76.3+ MB
None
